In [1]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32


In [2]:
!pip install rank_bm25 #키워드 기반의 랭킹 알고리즘

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00


In [8]:
# 기존 설치 제거 + 캐시 클린
!pip uninstall -y faiss-gpu faiss-cpu > /dev/null 2>&1
!rm -rf /root/.cache/pip

In [9]:
# NVIDIA 공식 채널을 통한 CUDA 12.4 설치
!pip install faiss-gpu-cu12 --extra-index-url=https://pypi.nvidia.com --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 109.9 MB/s eta 0:00:00


In [5]:
from langchain.retrievers import EnsembleRetriever # 여러 retriever를 입력으로 받아 처리
from langchain_community.retrievers import BM25Retriever  #TF-IDF 계열의 검색 알고리즘
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [10]:
doc_list_1 = [
    "사과: 달콤하고 아삭한 맛이 특징이며, 다양한 요리에 사용됩니다.",
    "바나나: 부드럽고 달콤하며, 에너지를 빠르게 공급해줍니다.",
    "포도: 신맛과 단맛이 조화를 이루며, 항산화 효과가 있습니다.",
    "오렌지: 비타민 C가 풍부하여 면역력 강화에 좋습니다.",
    "수박: 수분이 많아 더운 날씨에 시원하게 먹기 좋습니다.",
    "망고: 열대과일로서 달콤하고 부드러운 식감을 가지고 있습니다."
]

bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

doc_list_2 = [
    "사과: 다양한 색깔이 있으며, 특히 빨간 사과는 단맛이 강합니다.",
    "바나나: 껍질이 노랗고 익으면 더욱 달콤해집니다.",
    "포도: 적포도와 청포도로 나뉘며, 와인의 주 재료로도 사용됩니다.",
    "오렌지: 즙이 많아 주스로도 많이 활용됩니다.",
    "수박: 껍질이 두껍고 속이 붉거나 노란색을 띱니다.",
    "망고: 익을수록 향이 강해지며, 말려서도 많이 섭취됩니다."
]

embedding = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key="각자의 api입력")
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [11]:
query = "항산화 효과가 있는 과일은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

<ipython-input-11-dcd13fe278fc>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ensemble_result = ensemble_retriever.get_relevant_documents(query)


[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='포도: 신맛과 단맛이 조화를 이루며, 항산화 효과가 있습니다.'), Document(id='6a3ccedb-de43-427e-915c-3ae1defb078f', metadata={'source': 2}, page_content='사과: 다양한 색깔이 있으며, 특히 빨간 사과는 단맛이 강합니다.'), Document(metadata={'source': 1}, page_content='망고: 열대과일로서 달콤하고 부드러운 식감을 가지고 있습니다.'), Document(id='78afd2ef-1f4b-4851-a25b-2bdd4ab8fb24', metadata={'source': 2}, page_content='망고: 익을수록 향이 강해지며, 말려서도 많이 섭취됩니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='포도: 신맛과 단맛이 조화를 이루며, 항산화 효과가 있습니다.'), Document(metadata={'source': 1}, page_content='망고: 열대과일로서 달콤하고 부드러운 식감을 가지고 있습니다.')]

[FAISS Retriever]
 [Document(id='6a3ccedb-de43-427e-915c-3ae1defb078f', metadata={'source': 2}, page_content='사과: 다양한 색깔이 있으며, 특히 빨간 사과는 단맛이 강합니다.'), Document(id='78afd2ef-1f4b-4851-a25b-2bdd4ab8fb24', metadata={'source': 2}, page_content='망고: 익을수록 향이 강해지며, 말려서도 많이 섭취됩니다.')]



In [12]:
query = "건조해서 먹을 수 있는 과일은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='망고: 열대과일로서 달콤하고 부드러운 식감을 가지고 있습니다.'), Document(id='6a3ccedb-de43-427e-915c-3ae1defb078f', metadata={'source': 2}, page_content='사과: 다양한 색깔이 있으며, 특히 빨간 사과는 단맛이 강합니다.'), Document(metadata={'source': 1}, page_content='수박: 수분이 많아 더운 날씨에 시원하게 먹기 좋습니다.'), Document(id='810dfce0-c966-426c-ba77-3f3a7bd21f1a', metadata={'source': 2}, page_content='바나나: 껍질이 노랗고 익으면 더욱 달콤해집니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='망고: 열대과일로서 달콤하고 부드러운 식감을 가지고 있습니다.'), Document(metadata={'source': 1}, page_content='수박: 수분이 많아 더운 날씨에 시원하게 먹기 좋습니다.')]

[FAISS Retriever]
 [Document(id='6a3ccedb-de43-427e-915c-3ae1defb078f', metadata={'source': 2}, page_content='사과: 다양한 색깔이 있으며, 특히 빨간 사과는 단맛이 강합니다.'), Document(id='810dfce0-c966-426c-ba77-3f3a7bd21f1a', metadata={'source': 2}, page_content='바나나: 껍질이 노랗고 익으면 더욱 달콤해집니다.')]

